In [38]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import tmdbsimple

In [2]:
data_pathway = './data/ratings.csv'

data = pd.read_csv(data_pathway)  

In [103]:
def preprocess_data(data):
    # Drop rows with NaN entries
    processed_data = data.dropna()

    # Converts Year column from float to int
    processed_data = processed_data.astype({'Year':'int'})

    
    return processed_data

def extract_TMDb_data(data):
    # We'll use the URI column to get to the TMDb website to scrape the necessary data
    URI_list = data["Letterboxd URI"]
    TMDb_links = []

    for URI in URI_list:
        page = requests.get(URI)

        soup = BeautifulSoup(page.content, "html.parser")

        results = soup.find(class_="col-17")
        sections = results.find_all("section", class_="section col-10 col-main")

        for element in sections:
            paragraphs = element.find_all("p", class_="text-link text-footer")
            for hyperlink in paragraphs:
                links = hyperlink.find_all("a", class_="micro-button track-event")
                for link in links:
                    if link.text == "TMDb":
                        TMDb_links.append(link["href"])

    runtime = []
    genres = []
    keywords = []
    budget = []
    revenue = []
    original_language = []
    content_score = []

    for TMDb_link in TMDb_links:
        page = requests.get(TMDb_link)
        
        soup = BeautifulSoup(page.content, "html.parser")

        

    return TMDb_links


In [104]:
processed_data = preprocess_data(data)

TMDb_links = pd.DataFrame(extract_TMDb_data(processed_data))

In [122]:
data

,Date,Name,Year,Letterboxd URI,Rating
0,2020-09-11,Knives Out,2019.0,https://boxd.it/jWEA,4.0
1,2020-09-11,Inception,2010.0,https://boxd.it/1skk,4.0
2,2020-09-11,Spider-Man: Into the Spider-Verse,2018.0,https://boxd.it/azpY,3.5
3,2020-09-11,Avengers: Infinity War,2018.0,https://boxd.it/9vEe,3.5
4,2020-09-11,Guardians of the Galaxy,2014.0,https://boxd.it/3VH2,3.5
...,...,...,...,...,...
386,2023-07-18,Joy Ride,2023.0,https://boxd.it/wEeK,3.0
387,2023-07-20,The Story of Stuff,2007.0,https://boxd.it/4JBI,4.5
388,2023-07-22,Barbie,2023.0,https://boxd.it/bCLK,3.5
389,2023-07-23,My Neighbor Totoro,1988.0,https://boxd.it/20eA,3.0


In [128]:
page = requests.get("https://www.themoviedb.org/movie/546554/")
print(page)

soup = BeautifulSoup(page.content, "html.parser")
print(soup)



<Response [403]>
<!DOCTYPE html>

<html class="no-js" lang="en">
<head>
<title>Request Error (403) - The Movie Database (TMDb)</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="on" http-equiv="cleartype"/>
<meta charset="utf-8"/>
<meta content="noindex" name="robots"/>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="True" name="HandheldFriendly"/>
<meta content="320" name="MobileOptimized"/>
<meta content="width=1120" name="viewport"/>
<meta content="#032541" name="theme-color"/>
<link href="/assets/2/apple-touch-icon-57ed4b3b0450fd5e9a0c20f34e814b82adaa1085c79bdde2f00ca8787b63d2c4.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/assets/2/favicon-32x32-543a21832c8931d3494a68881f6afcafc58e96c5d324345377f3197a37b367b5.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/assets/2/favicon-16x16-b362d267873ce9c5a39f686a11fe67fec2a72ed25fa8396c11b71aa43c938b11.png" rel=

In [ ]:
results = soup.find(class_="col-17")
sections = results.find_all("section", class_="section col-10 col-main")

<a class="micro-button track-event" data-track-action="IMDb" href="http://www.imdb.com/title/tt8946378/maindetails" target="_blank">IMDb</a>
<a class="micro-button track-event" data-track-action="TMDb" href="https://www.themoviedb.org/movie/546554/" target="_blank">TMDb</a>
https://www.themoviedb.org/movie/546554/
